# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
output_data_file = "output_data/cities.csv"
output_data_df = pd.read_csv(output_data_file)
output_data_df



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Severo-Kuril'sk,50.6789,156.1250,34.21,96,100,16.17,RU,1619573277
1,1,Biak,-0.9131,122.8766,82.44,69,7,3.58,ID,1619573277
2,2,Mahébourg,-20.4081,57.7000,77.00,88,75,11.50,MU,1619573278
3,3,Hermanus,-34.4187,19.2345,53.01,88,86,4.56,ZA,1619573278
4,4,Vaini,-21.2000,-175.2000,75.20,83,75,12.66,TO,1619573279
...,...,...,...,...,...,...,...,...,...,...
562,562,Tāybād,34.7400,60.7756,72.70,16,6,5.77,IR,1619573446
563,563,Bubaque,11.2833,-15.8333,78.49,65,100,17.40,GW,1619573446
564,564,Dunmore East,52.1514,-6.9861,46.99,91,50,3.00,IE,1619573447
565,565,Mitú,1.1983,-70.1733,71.26,99,93,2.37,CO,1619573447


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
locations = output_data_df[["Lat", "Lng"]]

humidity = output_data_df['Humidity']


In [32]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#max_temp= > 80 degrees but < 70
output_data_df.drop(columns=['Unnamed: 0'], inplace = True)
max_temp_df=output_data_df.loc[(output_data_df ['Max Temp']<=80)&(output_data_df ['Max Temp']>=70)]
max_temp_df=max_temp_df.loc[(max_temp_df ['Wind Speed']<10)]
max_temp_df=max_temp_df.loc[(max_temp_df ['Cloudiness']==0)]


In [45]:
max_temp_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,São Filipe,14.8961,-24.4956,72.64,76,0,7.76,CV,1619573294
60,Carnarvon,-24.8667,113.6333,78.80,36,0,8.05,AU,1619573295
79,Cayenne,4.9333,-52.3333,73.99,94,0,2.30,GF,1619573206
142,Saint-Philippe,-21.3585,55.7679,75.20,83,0,6.91,RE,1619573318
270,Inhambane,-23.8650,35.3833,72.03,85,0,8.59,MZ,1619573357
374,Saint-Pierre,-21.3393,55.4781,75.20,83,0,6.91,RE,1619573388
378,Oussouye,12.4850,-16.5469,73.40,73,0,4.61,SN,1619573389
508,Conceição do Araguaia,-8.2578,-49.2647,77.00,88,0,2.30,BR,1619573430
512,Chardara,41.2547,67.9692,70.23,41,0,3.76,KZ,1619573432
534,Saint-Joseph,-21.3667,55.6167,75.20,83,0,6.91,RE,1619573438


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
hotel_df=max_temp_df

In [48]:
hotel_df['Hotel Name']=""


<ipython-input-48-af7c5f0ffd36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
58,São Filipe,14.8961,-24.4956,72.64,76,0,7.76,CV,1619573294,
60,Carnarvon,-24.8667,113.6333,78.80,36,0,8.05,AU,1619573295,
79,Cayenne,4.9333,-52.3333,73.99,94,0,2.30,GF,1619573206,
142,Saint-Philippe,-21.3585,55.7679,75.20,83,0,6.91,RE,1619573318,
270,Inhambane,-23.8650,35.3833,72.03,85,0,8.59,MZ,1619573357,
374,Saint-Pierre,-21.3393,55.4781,75.20,83,0,6.91,RE,1619573388,
378,Oussouye,12.4850,-16.5469,73.40,73,0,4.61,SN,1619573389,
508,Conceição do Araguaia,-8.2578,-49.2647,77.00,88,0,2.30,BR,1619573430,
512,Chardara,41.2547,67.9692,70.23,41,0,3.76,KZ,1619573432,
534,Saint-Joseph,-21.3667,55.6167,75.20,83,0,6.91,RE,1619573438,


In [49]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
58,São Filipe,14.8961,-24.4956,72.64,76,0,7.76,CV,1619573294,
60,Carnarvon,-24.8667,113.6333,78.80,36,0,8.05,AU,1619573295,
79,Cayenne,4.9333,-52.3333,73.99,94,0,2.30,GF,1619573206,
142,Saint-Philippe,-21.3585,55.7679,75.20,83,0,6.91,RE,1619573318,
270,Inhambane,-23.8650,35.3833,72.03,85,0,8.59,MZ,1619573357,
374,Saint-Pierre,-21.3393,55.4781,75.20,83,0,6.91,RE,1619573388,
378,Oussouye,12.4850,-16.5469,73.40,73,0,4.61,SN,1619573389,
508,Conceição do Araguaia,-8.2578,-49.2647,77.00,88,0,2.30,BR,1619573430,
512,Chardara,41.2547,67.9692,70.23,41,0,3.76,KZ,1619573432,
534,Saint-Joseph,-21.3667,55.6167,75.20,83,0,6.91,RE,1619573438,


In [68]:
#airport_ratings
params = {
    "radius": 5000,
    "types": "hotels",
    "key": g_key
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in max_temp_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    hotel_info = requests.get(base_url, params=params)
    hotel_info = hotel_info.json()
    try:
        hotel_df.loc[index,'Hotel Name'] = hotel_info['results'][0]['name']
    except:
        print('Error')
        pass

C:\Users\anici\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [69]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
58,São Filipe,14.8961,-24.4956,72.64,76,0,7.76,CV,1619573294,Sao Filipe
60,Carnarvon,-24.8667,113.6333,78.80,36,0,8.05,AU,1619573295,Carnarvon
79,Cayenne,4.9333,-52.3333,73.99,94,0,2.30,GF,1619573206,Cayenne
142,Saint-Philippe,-21.3585,55.7679,75.20,83,0,6.91,RE,1619573318,Saint-Philippe
270,Inhambane,-23.8650,35.3833,72.03,85,0,8.59,MZ,1619573357,Maxixe
374,Saint-Pierre,-21.3393,55.4781,75.20,83,0,6.91,RE,1619573388,Saint Pierre
378,Oussouye,12.4850,-16.5469,73.40,73,0,4.61,SN,1619573389,Oussouye
508,Conceição do Araguaia,-8.2578,-49.2647,77.00,88,0,2.30,BR,1619573430,Conceição do Araguaia
512,Chardara,41.2547,67.9692,70.23,41,0,3.76,KZ,1619573432,Shardara
534,Saint-Joseph,-21.3667,55.6167,75.20,83,0,6.91,RE,1619573438,Saint-Joseph


In [59]:
#pprint(hotel_info['results'][0]['name'])

'Umluj'


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [75]:
# Add marker layer ontop of heat map
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]



# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))